<a href="https://colab.research.google.com/github/bnsreenu/python_for_microscopists/blob/master/311_fine_tuning_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://youtu.be/nsdCRVuprDY

In [ ]:
!pip install transformers

In [ ]:
!pip install -U PyPDF2
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 11.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import os
import docx

In [ ]:
# Functions to read different file types
def read_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".pdf"):
            combined_text += read_pdf(file_path)
        elif filename.endswith(".docx"):
            combined_text += read_word(file_path)
        elif filename.endswith(".txt"):
            combined_text += read_txt(file_path)
        elif filename.endswith(".json"):
            combined_text += read_txt(file_path)
    return combined_text


In [ ]:
!mkdir dataset

In [ ]:
# Read documents from the directory
#train_directory = '/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/training_data/full_text'
train_directory = '/content/dataset'
text_data = read_documents_from_directory(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters


## Main Traning Code

In [2]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [3]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [4]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [5]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  # trainer.train()
  trainer.train("checkpoint-9500")
  trainer.save_model()

In [6]:
!mkdir output_dir

In [20]:

#train_file_path = "/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/combined_text/full_text/train.txt"
train_file_path = "/content/DATA SCIENTIST INTERVIEW QUESTIONS .txt"
model_name = 'gpt2-medium'
#output_dir = '/content/drive/MyDrive/ColabNotebooks/models/chat_models/custom_full_text'
output_dir = '/content/output_dir'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 15
save_steps = 500

In [10]:
import wandb
wandb.init(mode='disabled')

In [21]:
# Train
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)
# learning_rate = 3e-5  # Adjust based on your needs

# train(
#     train_file_path=train_file_path,
#     model_name=model_name,
#     output_dir=output_dir,
#     overwrite_output_dir=overwrite_output_dir,
#     per_device_train_batch_size=per_device_train_batch_size,
#     num_train_epochs=num_train_epochs,
#     save_steps=save_steps,
#     save_total_limit=3
# )


/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


## Training with Checkpoints

In [32]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  # trainer.train()
  trainer.train("/content/output_dir/checkpoint-105")
  trainer.save_model()

In [33]:
train_file_path = "/content/DATA SCIENTIST INTERVIEW QUESTIONS .txt"
model_name = 'gpt2-medium'
#output_dir = '/content/drive/MyDrive/ColabNotebooks/models/chat_models/custom_full_text'
output_dir = '/content/output_dir'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 25
save_steps = 500

In [34]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Step,Training Loss


## Predictions

Inference

In [35]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [36]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

This model got trained on the entire text and took much longer to train, and yet it fails to give meaningful results.

In [37]:
model1_path = "/content/output_dir"
sequence1 = '''difference between supervised and
unsupervised learning.'''
max_len = 50
generate_text(model1_path, sequence1, max_len)

difference between supervised and
unsupervised learning.

What is the difference between categorical
Q.65
and unconfined variables in the context of dimensionality
reduction?
Categorical variables can be categorized into



In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    save_total_limit=3,  # Keep only the last 3 checkpoints
    learning_rate=learning_rate
)

In [ ]:
import shutil

# Define the folder path and output zip file name
folder_path = "/content/output_dir/checkpoint-9000"  # Replace with the folder you want to zip
output_zip = "your_zip_file"  # Name of the output ZIP file (without .zip extension)

# Create a zip file
shutil.make_archive(output_zip, 'zip', folder_path)

print(f"Folder '{folder_path}' successfully zipped as '{output_zip}.zip'")

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps,
          learning_rate, save_total_limit):

    # Load tokenizer and dataset
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    train_dataset = load_dataset(train_file_path, tokenizer)  # Ensure load_dataset() is defined properly
    data_collator = load_data_collator(tokenizer)  # Ensure load_data_collator() is defined properly

    tokenizer.save_pretrained(output_dir)

    # Load model
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.save_pretrained(output_dir)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        save_steps=save_steps,
        learning_rate=learning_rate,  # ✅ Fixed learning rate parameter
        save_total_limit=save_total_limit  # ✅ Limit number of saved checkpoints
    )

    # Define trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    # Train and save the model
    trainer.train()
    trainer.save_model()

# Set parameters
learning_rate = 3e-5
save_total_limit = 3

# Train model
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    learning_rate=learning_rate,
    save_total_limit=save_total_limit
)


## Converting it into zip file

In [ ]:
import shutil

def zip_folder(folder_path, output_path):
  """Zips a folder to the specified output path.

  Args:
    folder_path: The path to the folder you want to zip.
    output_path: The path to the output zip file.
  """
  shutil.make_archive(output_path, 'zip', folder_path)

# Example usage:
folder_to_zip = '/content/output_dir'  # Replace with your folder path
output_zip_file = '/content/my_folder.zip'  # Replace with desired output path
zip_folder(folder_to_zip, output_zip_file)

In [ ]:
text2 = "how ai will change the future of tech recruitment.\nHow? By understanding AI and automation, Aja can help you make informed decisions by helping AI assist recruiters with their initial resume processing, converting incomplete resumes into complete pieces of information and highlighting skills using AI-recommended skills.\n3. Building Confidence and a Potential Job Search\nAI can change the hiring process for the worse. Confidence and a job search are the things that attract recruiters most.\nAI can help you build a strong employer's confidence through various stages of the hiring process, from preparing candidate profiles to evaluating and promoting candidates.\n4. Enhanced Job Search Experience\nAI can enhance the job search experience for those looking for a specific role and the pathway to that specific role. AI supports job seekers by analyzing market trends, uncovering skills gaps, and highlighting skills with exceptional performances.\n5. Enhanced Candidate Experience\nAI supports job seekers by analyzing skills using AI-recommended skills, highlighting skills using AI-recommended skills, and more.\n6. Enhanced Candidate Experience\nAI supports job seekers by analyzing skills using AI-recommended skills, highlighting skills using AI-recommended skills, and more.\n7. Enhanced Candidate Experience Plus\nAI supports job seekers by analyzing skills using AI-recommended skills, highlighting skills using AI-recommended skills, and more.\nConclusion: Continuous Learning and Upskilling with AI in Recruitment is Ease\nAs recruiters look for the best candidate, AI is giving its job to those candidates with the highest potential.\nIf you are a recruiter looking for a candidate with all the qualities of a potential job seeker, then Continuous Learning and Upskilling is the place to start.\nYou can start your journey by following the simple but effective ways AI supports job seekers by analyzing skills using AI-recommended skills, highlighting skills using AI-recommended skills, and more.\nIf you are a recruiter looking for candidates with all the attributes of a potential job seeker, then our AI-powered platform is for you.\nIntegrating innovative technologies like AI and automation is transforming the recruitment process for the better. Cloud Imperium now offers integrations with leading recruitment platforms like Remark Jobs, RemarkHR, and more. Register with the Remark Jobs App to build your professional resume today.          [][][][]"

In [ ]:
text = "why AI is important in present world but will not be in the future.\nAI: The Job Transformer\nIn the AI future, technologies like AI and automation play vital roles in revolutionising the work process and streamlining the hiring process. These technologies simplify many stages of the hiring process, from candidate sourcing to interview scheduling.\nAI and automation work on data-driven insights and subjective judgments, helping recruiters save time and reducing hiring biases from the initial resume shorting step. These technologies will evolve in the future. Hence, we can expect more innovative tools to emerge to make the recruitment process quicker, simpler, and more efficient.\n5. Prioritisation of Skills over Degree: Focus on Continuous Learning\nThe traditional practice of focusing on degrees and formal qualifications is shifting towards skills and continuous learning and upskilling. Employers now understand very well that the capacity to adapt and learn new skills and technologies is more critical than having a formal degree. The reason is the drastic and constant evolutions in technological advancement and the need for employees to continuously learn new skills and upgrade as per the industry/company requirements.\nThe reason is the drastic and constant evolutions in the recruitment industry and the need for employees to continuously learn new skills and upgrade as per the industry and company requirements. The employee-centric approach is starting from scratch to make the transition seamless and cost-effective.\n6. Empowering Diversity and Inclusion: For Inclusive Workplace\nDiversity and inclusion in the workplace have become essential for organisations globally. Therefore, organisations now focus on building inclusive workplaces where everyone feels valued, respected, and equal to witness the wonders of the work culture. The more inclusive the workplace becomes, the more essential the skills and cultural fit for everyone becomes. The more inclusive the workplace is among those who want to achieve equality, inclusion, and growth, the more essential the workplace will become for those seeking to leave their roles behind.\n7. Enhanced Hire Quality: For Inclusive Workplace\nThe quality of work depends on the number of employees. Hence, organisations now focus on building a strong employer brand and utilising existing strengths to retain and attract more employees. Providing employees with workplace training, development programs, and online learning platforms can help them hone their knowledge, skills, and habits to stay competitive in the changing work environment. These platforms can be your eyes and ears in the changing work environment, making it easier for you             [][][][][]"

In [ ]:

text_without_newline = text2.replace('\n', ' ')

print(text_without_newline)

how ai will change the future of tech recruitment. How? By understanding AI and automation, Aja can help you make informed decisions by helping AI assist recruiters with their initial resume processing, converting incomplete resumes into complete pieces of information and highlighting skills using AI-recommended skills. 3. Building Confidence and a Potential Job Search AI can change the hiring process for the worse. Confidence and a job search are the things that attract recruiters most. AI can help you build a strong employer's confidence through various stages of the hiring process, from preparing candidate profiles to evaluating and promoting candidates. 4. Enhanced Job Search Experience AI can enhance the job search experience for those looking for a specific role and the pathway to that specific role. AI supports job seekers by analyzing market trends, uncovering skills gaps, and highlighting skills with exceptional performances. 5. Enhanced Candidate Experience AI supports job se

In [ ]:
with open('blogs.txt','a') as f:
  f.write(text_without_newline)

In [ ]:
!python --version

Python 3.11.11


The following model was trained on 100 questions and answers based on the original text and it trained in a few seconds (50 epochs). It gives very meaningful results.

In [ ]:
model2_path = "/content/drive/MyDrive/ColabNotebooks/models/chat_models/custom_q_and_a"
sequence2 = "[Q] What is the Babel fish?"
max_len = 50
generate_text(model2_path, sequence2, max_len)

[Q] What is the Babel fish?
[A] The Babel fish is a small, yellow, leech-like creature that can be inserted into a person's ear to instantly translate any language into the user's native tongue.
[


In [ ]:
pip install librosa numpy scipy noisereduce soundfile


In [ ]:
pip install librosa numpy scipy noisereduce soundfile pydub


In [ ]:
import librosa
import noisereduce as nr
import numpy as np
import soundfile as sf
from pydub import AudioSegment

# Convert MP3 to WAV
mp3_file = "/content/Standard recording 244.mp3"
wav_file = "converted.wav"
audio = AudioSegment.from_mp3(mp3_file)
audio.export(wav_file, format="wav")

# Load WAV file
y, sr = librosa.load(wav_file, sr=None)

# Perform noise reduction
reduced_noise = nr.reduce_noise(y=y, sr=sr, prop_decrease=0.8)

# Save output file
output_file = "output.wav"
sf.write(output_file, reduced_noise, sr)

print("Background noise removed successfully. Output saved as", output_file)


Background noise removed successfully. Output saved as output.wav


In [ ]:
pip install ffmpeg-python


In [ ]:
import ffmpeg

input_file = "/content/Standard recording 244.mp3"
output_file = "output1.mp3"

# Apply noise reduction using ffmpeg
ffmpeg.input(input_file).output(output_file, af="afftdn").run()

print("Noise reduction complete. Saved as:", output_file)


Noise reduction complete. Saved as: output1.mp3
